<a href="https://colab.research.google.com/github/kavindi1996/KavindiKaluarachchi/blob/main/Football_match_results_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from time import time
from sklearn.metrics import f1_score
from os import path, makedirs, walk
from joblib import dump, load
import json
import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('DATASET.csv')

In [ ]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,Unnamed: 71,Unnamed: 72,Unnamed: 73
0,E0,8/15/2009,Aston Villa,Wigan,0.0,2.0,A,0.0,1.0,A,...,1.81,21.0,0.0,1.28,1.22,4.40,3.99,NaN,NaN,NaN
1,E0,8/15/2009,Blackburn,Man City,0.0,2.0,A,0.0,1.0,A,...,1.71,24.0,0.0,2.58,2.38,1.60,1.54,NaN,NaN,NaN
2,E0,8/15/2009,Bolton,Sunderland,0.0,1.0,A,0.0,1.0,A,...,1.66,23.0,0.0,1.68,1.61,2.33,2.23,NaN,NaN,NaN
3,E0,8/15/2009,Chelsea,Hull,2.0,1.0,H,1.0,1.0,D,...,2.26,22.0,0.0,1.03,1.02,17.05,12.96,NaN,NaN,NaN
4,E0,8/15/2009,Everton,Arsenal,1.0,6.0,A,0.0,3.0,A,...,1.70,24.0,0.0,2.27,2.20,1.73,1.63,NaN,NaN,NaN


In [ ]:
raw_match_stats = df[[
 'Date',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'HC',
 'AC',
 'AS',
 'HS',
 'HST',
 'AST',
 'HF',
 'AF',
 'HY',
 'AY',]]

In [ ]:
raw_match_stats = raw_match_stats.dropna()

In [ ]:
raw_match_stats


,Date,HomeTeam,AwayTeam,FTHG,FTAG,HC,AC,AS,HS,HST,AST,HF,AF,HY,AY
0,8/15/2009,Aston Villa,Wigan,0.0,2.0,4.0,6.0,14.0,11.0,5.0,7.0,15.0,14.0,2.0,2.0
1,8/15/2009,Blackburn,Man City,0.0,2.0,5.0,4.0,8.0,17.0,9.0,5.0,12.0,9.0,2.0,1.0
2,8/15/2009,Bolton,Sunderland,0.0,1.0,4.0,7.0,20.0,11.0,3.0,13.0,16.0,10.0,2.0,1.0
3,8/15/2009,Chelsea,Hull,2.0,1.0,12.0,4.0,7.0,26.0,12.0,3.0,13.0,15.0,1.0,2.0
4,8/15/2009,Everton,Arsenal,1.0,6.0,4.0,9.0,15.0,8.0,5.0,9.0,11.0,13.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18255,18/05/2019,Levante,Ath Madrid,2.0,2.0,4.0,0.0,7.0,17.0,8.0,9.0,7.0,6.0,1.0,0.0
18256,18/05/2019,Sevilla,Ath Bilbao,2.0,0.0,3.0,4.0,3.0,9.0,1.0,14.0,19.0,2.0,2.0,0.0
18257,18/05/2019,Valladolid,Valencia,0.0,2.0,2.0,1.0,4.0,9.0,6.0,12.0,10.0,8.0,3.0,0.0
18258,19/05/2019,Eibar,Barcelona,2.0,2.0,2.0,4.0,8.0,6.0,3.0,11.0,3.0,5.0,2.0,0.0


In [ ]:
raw_match_stats.loc[raw_match_stats['FTHG'] == raw_match_stats['FTAG'], 'ATR'] = 'D'
raw_match_stats.loc[raw_match_stats['FTHG'] > raw_match_stats['FTAG'], 'ATR'] = 'L'
raw_match_stats.loc[raw_match_stats['FTHG'] < raw_match_stats['FTAG'], 'ATR'] = 'W'

In [ ]:
raw_match_stats


,Date,HomeTeam,AwayTeam,FTHG,FTAG,HC,AC,AS,HS,HST,AST,HF,AF,HY,AY,ATR
0,8/15/2009,Aston Villa,Wigan,0.0,2.0,4.0,6.0,14.0,11.0,5.0,7.0,15.0,14.0,2.0,2.0,W
1,8/15/2009,Blackburn,Man City,0.0,2.0,5.0,4.0,8.0,17.0,9.0,5.0,12.0,9.0,2.0,1.0,W
2,8/15/2009,Bolton,Sunderland,0.0,1.0,4.0,7.0,20.0,11.0,3.0,13.0,16.0,10.0,2.0,1.0,W
3,8/15/2009,Chelsea,Hull,2.0,1.0,12.0,4.0,7.0,26.0,12.0,3.0,13.0,15.0,1.0,2.0,L
4,8/15/2009,Everton,Arsenal,1.0,6.0,4.0,9.0,15.0,8.0,5.0,9.0,11.0,13.0,0.0,0.0,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18255,18/05/2019,Levante,Ath Madrid,2.0,2.0,4.0,0.0,7.0,17.0,8.0,9.0,7.0,6.0,1.0,0.0,D
18256,18/05/2019,Sevilla,Ath Bilbao,2.0,0.0,3.0,4.0,3.0,9.0,1.0,14.0,19.0,2.0,2.0,0.0,L
18257,18/05/2019,Valladolid,Valencia,0.0,2.0,2.0,1.0,4.0,9.0,6.0,12.0,10.0,8.0,3.0,0.0,W
18258,19/05/2019,Eibar,Barcelona,2.0,2.0,2.0,4.0,8.0,6.0,3.0,11.0,3.0,5.0,2.0,0.0,D


In [ ]:
# Split the raw_match_stats to two datasets (home_team_stats and away_team_stats)

home_team_stats = raw_match_stats[[
 'Date',
 'HomeTeam',
 'FTHG',
 'HC',
 'HS',
 'HST',
 'HF',
 ]]

away_team_stats = raw_match_stats[[
 'Date',
 'AwayTeam',
 'FTAG',
 'AC',
 'AS',
 'AST',
 'AF',
 ]]



# stack these two datasets so that each row is the stats for a team for one match (team_stats_per_match)
team_stats_per_match = home_team_stats.append(away_team_stats)

In [ ]:
# At each row of this dataset, get the team name, find the stats for that team during the last 5 matches, and average these stats (avg_stats_per_team). 

avg_stat_columns = ['goals_per_match','corners_per_match','shots_per_match','shotsOnTarget_per_match','fouls_per_match']
stats_list = []
for index, row in team_stats_per_match.iterrows():
    team_stats_last_five_matches = team_stats_per_match.loc[(team_stats_per_match['HomeTeam']==row['AwayTeam']) & (team_stats_per_match['Date']<row['Date'])].sort_values(by=['Date'], ascending=False)
    stats_list.append(team_stats_last_five_matches.iloc[0:5,:].mean(axis=0).values[0:5])

avg_stats_per_team = pd.DataFrame(stats_list, columns=avg_stat_columns)

In [ ]:
# Add these stats to the team_stats_per_match dataset.

team_stats_per_match = pd.concat([team_stats_per_match.reset_index(drop=True), avg_stats_per_team], axis=1, ignore_index=False)

In [ ]:
# Re-segment the home and away teams.

home_team_stats = team_stats_per_match.iloc[:int(team_stats_per_match.shape[0]/2),:]
away_team_stats = team_stats_per_match.iloc[int(team_stats_per_match.shape[0]/2):,:]

home_team_stats.columns = ['HomeTeam_'+str(col) for col in home_team_stats.columns]
away_team_stats.columns = ['AwayTeam_'+str(col) for col in away_team_stats.columns]

In [ ]:
# Combine at each match to get a dataset with a row representing each match. 
# drop the NA rows (earliest match for each team, i.e no previous stats)

match_stats = pd.concat([home_team_stats, away_team_stats.reset_index(drop=True)], axis=1, ignore_index=False)
match_stats = match_stats.dropna().reset_index(drop=True)

In [ ]:
# create columns with average stat differences between the two teams

match_stats['goals_per_match_diff'] = (match_stats['HomeTeam_goals_per_match'] - match_stats['AwayTeam_goals_per_match'])
match_stats['corners_per_match_diff'] = (match_stats['HomeTeam_corners_per_match'] - match_stats['AwayTeam_corners_per_match'])
match_stats['shots_per_match_diff'] = (match_stats['HomeTeam_shots_per_match'] - match_stats['AwayTeam_shots_per_match'])
match_stats['shotsOnTarget_per_match_diff'] = (match_stats['HomeTeam_shotsOnTarget_per_match'] - match_stats['AwayTeam_shotsOnTarget_per_match'])
match_stats['fouls_per_match_diff'] = (match_stats['HomeTeam_fouls_per_match'] - match_stats['AwayTeam_fouls_per_match'])


In [ ]:
match_stats

,HomeTeam_Date,HomeTeam_HomeTeam,HomeTeam_FTHG,HomeTeam_HC,HomeTeam_HS,HomeTeam_HST,HomeTeam_HF,HomeTeam_AwayTeam,HomeTeam_FTAG,HomeTeam_AC,...,AwayTeam_goals_per_match,AwayTeam_corners_per_match,AwayTeam_shots_per_match,AwayTeam_shotsOnTarget_per_match,AwayTeam_fouls_per_match,goals_per_match_diff,corners_per_match_diff,shots_per_match_diff,shotsOnTarget_per_match_diff,fouls_per_match_diff


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score, confusion_matrix, roc_auc_score, classification_report, log_loss

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

In [ ]:
target = match_stats[['HomeTeam_FTHG']].replace(['W','L','D'],[0,1,2])

features = match_stats[['goals_per_match_diff', 'corners_per_match_diff',
       'shots_per_match_diff', 'shotsOnTarget_per_match_diff',
       'fouls_per_match_diff']]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(features, target, test_size=0.2, stratify = target)

ValueError: ignored